<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines e Ajuste com Cross-Validation com Spark</h1></center>  

**Ajuste de Hiper Parâmetros:**

> + Um classificador pode ter diversos hiper parâmetros
> + A configuração dos hiper parâmetros as vezes não é intuitiva e nem “cientifica”
> + A combinação de muitos hiper parâmetros pode ser gigantesca
> + Inviável testar manualmente todas as combinações

> + Spark facilita o teste de diferentes hiper parâmetros
> + Você pode testar diferentes configurações para todo o Pipeline
> + ParamGridBuilder: você configura quais parâmetros e quais domínios quer testar
> + Utiliza Cross-Validation ou Train-Validation Split


<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [3]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Pepilines_tune").getOrCreate()

---

<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines com Spark com Multi Layers Perceptron usando o DataSet Íris</h1></center>




<center> <h2 style="background-color:DarkKhaki; color:white" >Carregando o Cunjunto de Dados Íris</h2></center> 

In [4]:
iris = spark.read.csv("../Material_do_Curso/iris.csv", header=True, inferSchema=True, sep=",")
iris.show(5)

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



<center> <h2 style="background-color:DarkKhaki; color:white" >Separando os dados entre conjunto de traino e teste</h2></center> 

In [5]:
iris_train, iris_test = iris.randomSplit([0.8, 0.2])


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Colocando os dados no formato aceito pelo Spark</h3></center>

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth",
                                    "petallength", "petalwidth"], outputCol="independente")


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Convertendo um tipo categórico em numérico com StringIndexer</center>

In [12]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")

---

<center> <h2 style="background-color:DarkKhaki; color:white" >Importação do Módulo do PySpark Para Criação do Modelo de Rde Neural Artificial</h2></center>

In [13]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3],
                                     featuresCol="independente", labelCol="label")

---

<center> <h1 style="background-color:seagreen; color:white" >Criando o Pipeline e Aplicando Cross-Validation</h1></center> 

 #### Cross-Validation
> + Divide os dados de treino em partições (folds) que são usadas para treino (2/3 dos dados) e teste (1/3 dos dados)
> + 3 Partições = 3 conjuntos de treino e 3 de teste
> + As partições são treinadas e testadas nos domínios do grid de hiper parâmetros fornecido
> + Quando encontrar o melhor conjunto de hiper parâmetros, ele novamente cria o modelo usando todos os dados de treino


<br>



In [14]:
from pyspark.ml import Pipeline

In [15]:
pipeline = Pipeline(stages=[vector, indexer, mlp])


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Aplicando o Cross-Validation</center>

In [17]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
performace = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000])
grid = grid.addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]])
grid = grid.build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=grid,
                          evaluator=performace,
                          numFolds=3)

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Executando o Pipiline com o Tune do Modelo</center>

In [19]:
modelo = crossval.fit(iris_train)

23/03/05 22:54:44 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:54:45 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:54:45 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:54:45 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/03/05 22:54:45 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/03/05 22:54:45 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.09375
23/03/05 22:54:45 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
23/03/05 22:54:55 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:54:56 E

23/03/05 22:55:38 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
23/03/05 22:55:40 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
23/03/05 22:55:58 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:55:58 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/03/05 22:55:58 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/03/05 22:55:59 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.09375
23/03/05 22:55:59 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Teste do Modelo usando o conjunto teste</center>

In [20]:
previsao = modelo.transform(iris_test)

In [21]:
previsao.select("independente", "label",
                "prediction", "rawprediction",
                "probability").show(10)

+-----------------+-----+----------+--------------------+--------------------+
|     independente|label|prediction|       rawprediction|         probability|
+-----------------+-----+----------+--------------------+--------------------+
|[4.4,3.2,1.3,0.2]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[4.6,3.2,1.4,0.2]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[4.6,3.4,1.4,0.3]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[4.8,3.4,1.6,0.2]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[4.9,2.4,3.3,1.0]|  0.0|       0.0|[256.361767431977...|[1.0,6.6073272808...|
|[4.9,3.1,1.5,0.1]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[4.9,3.1,1.5,0.1]|  2.0|       2.0|[91.117568027615,...|[5.37926516385694...|
|[5.0,2.0,3.5,1.0]|  0.0|       0.0|[256.361767431977...|[1.0,6.6073272808...|
|[5.0,2.3,3.3,1.0]|  0.0|       0.0|[256.361767431977...|[1.0,6.6073272808...|
|[5.0,3.2,1.2,0.2]|  2.0|       2.0|[91.117568027615


---

<center> <h1 style="background-color:seagreen; color:white" >Avaliação do Modelo usando PySpark</h1></center> 
<br>


In [24]:
performace = MulticlassClassificationEvaluator(labelCol="label",
                                               predictionCol="prediction",
                                               metricName="accuracy")
acuracia = performace.evaluate(previsao)
print(f"Acurácia do Modelo: {acuracia*100}%")

Acurácia do Modelo: 100.0%
